In [36]:
# Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz: datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz: (datetime.now(tz).replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz: (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz: (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz: (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-09-28T22:59:59.141779-05:00
morning time: 2023-09-29T09:00:00.142003-05:00
1695957707.142069


In [37]:
import requests
from const import GOOGLE_MAPS_API_KEY


COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
MY_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'
# travelTimes = {
#     now: lambda : datetime.datetime.now()
#     morning: lambda : datetime.datetime
# }


def computeDrivingRoute(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeDrivingRoute(MY_HOUSE, SQUARE_ONE_ADDRESS))

{
  "routes": [
    {
      "distanceMeters": 6812,
      "duration": "770s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@tA{EF]u@g@uAw@cB}AqBaCSIK?qCeDsCcDiDcE{@}@gFoGoBmC}GcIkEkFqCaDkCkDqJiL}@eAc@cAiJuKeDzEg@~@Of@qEnHyG|K_@Tw@fAkEbH}GzK]b@Kd@m@fA_KbPcC~DoCjEwF`JmAtAsAvAUHc@f@oDdFyCuDeBoBaGqGiLaNiFoGyCkDoRsU{LqOgIpOGRURGDgAD"
      }
    }
  ]
}



In [38]:


def computeTransitRoute(originAddress: str, destinationAddress: str, departureOption='now'):
    headers = {
        'content-type': 'application/json; application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeTransitRoute(MY_HOUSE, SQUARE_ONE_ADDRESS))

{
  "routes": [
    {
      "legs": [
        {
          "steps": [
            {},
            {},
            {
              "transitDetails": {
                "stopDetails": {
                  "arrivalStop": {
                    "name": "City Centre Transit Terminal Platform H",
                    "location": {
                      "latLng": {
                        "latitude": 43.593657,
                        "longitude": -79.646315
                      }
                    }
                  },
                  "arrivalTime": "2023-09-29T04:29:49Z",
                  "departureStop": {
                    "name": "The Credit Woodlands At Ballyclare Dr",
                    "location": {
                      "latLng": {
                        "latitude": 43.552136,
                        "longitude": -79.649819
                      }
                    }
                  },
                  "departureTime": "2023-09-29T04:04:26Z"
                },
            

In [39]:

def computeBicyleRoutes(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'BICYCLE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeBicyleRoutes(MY_HOUSE, SQUARE_ONE_ADDRESS))

{
  "routes": [
    {
      "distanceMeters": 7982,
      "duration": "1636s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@Qf@wDdGyArCs@`As@r@eBhAw@X}@TkB^gAb@_Ar@q@p@k@|@aAtB}@bCk@tBo@rDS^UNe@Fa@?[RQZGRe@hEGxA?h@HhAHbEIbBKv@_@vAeC`GQlAEv@@zAEdAQjAY|@s@jAmEtGaArAiBjCa@RuBX]Nm@`Au@tA_JcK_VqZAn@i@n@[z@IFe@Li@vAKt@Mf@_@`A]pBe@Ru@?q@Je@KKG[As@JuBE_BqBg@KQBILaApBu@w@gYwVcBiBu@iAiA}BkD{Ig@cAy@oAqEaGO]yC{DmAgBgAqAUM}AiBq@g@eFcCsC}@_Cc@kAMoDSiBWiBo@}@m@eAw@k@s@OK_C}Bo@[w@Yo@a@S[gAw@{@gAeGaHyBaC}@}@sGkHqCkCU_@eBcBw@cAp@aABQjBwCbKwRvAbB|AjBJ@Qx@"
      }
    }
  ]
}

